## IMPORT LIBRARIES

In [2]:
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd
import os
import cv2
import imutils
import random
from PIL import Image
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from PIL import Image
import matplotlib.pyplot as plt

## FUNCTIONS

In [3]:
def preprocess_image(image):
    if image is None or image.size == 0:
        return None
    # Redimensionar la imagen para mejorar la resolución
    resized = cv2.resize(image, None, fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)

    # Aplicar un enfoque para mejorar la nitidez
    sharpened = cv2.detailEnhance(resized, sigma_s=10, sigma_r=20)
    
    # Retornar la imagen preprocesada
    return sharpened

def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                         key=lambda b:b[1][i], reverse=reverse))
    return (cnts, boundingBoxes)

def get_numbers(image):
    
    letters = []
    
    if image is None or image.size == 0:
        print("Error: Empty image")
        return letters, None
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if len(cnts) == 0:
        print("Error: No contours found")
        return letters, None
    
    cnts = sort_contours(cnts, method="left-to-right")[0]
    
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            x, y, w, h = int(x), int(y), int(w), int(h) 
            roi = gray[y:y + h, x:x + w]
            thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            thresh = cv2.resize(thresh, (32, 32), interpolation=cv2.INTER_CUBIC)
            thresh = thresh.astype("float32") / 255.0
            thresh = np.expand_dims(thresh, axis=-1)
            thresh = thresh.reshape(1, 32, 32, 1)
            ypred = model_num.predict(thresh)
            ypred = LB_num.inverse_transform(ypred)
            [x] = ypred
            letters.append(x)
    return letters, image

def get_letters(image):
    
    letters = []
    
    if image is None or image.size == 0:
        print("Error: Empty image")
        return letters, None
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if len(cnts) == 0:
        print("Error: No contours found")
        return letters, None
    
    cnts = sort_contours(cnts, method="left-to-right")[0]
    
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            x, y, w, h = int(x), int(y), int(w), int(h) 
            roi = gray[y:y + h, x:x + w]
            thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            thresh = cv2.resize(thresh, (32, 32), interpolation=cv2.INTER_CUBIC)
            thresh = thresh.astype("float32") / 255.0
            thresh = np.expand_dims(thresh, axis=-1)
            thresh = thresh.reshape(1, 32, 32, 1)
            ypred = model_char.predict(thresh)
            ypred = LB_char.inverse_transform(ypred)
            [x] = ypred
            letters.append(x)
    return letters, image

def get_combined(image):
    
    letters = []
    
    if image is None or image.size == 0:
        print("Error: Empty image")
        return letters, None
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if len(cnts) == 0:
        print("Error: No contours found")
        return letters, None
    
    cnts = sort_contours(cnts, method="left-to-right")[0]
    
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            x, y, w, h = int(x), int(y), int(w), int(h) 
            roi = gray[y:y + h, x:x + w]
            thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            thresh = cv2.resize(thresh, (32, 32), interpolation=cv2.INTER_CUBIC)
            thresh = thresh.astype("float32") / 255.0
            thresh = np.expand_dims(thresh, axis=-1)
            thresh = thresh.reshape(1, 32, 32, 1)
            ypred = model_gen.predict(thresh)
            ypred = LB_combined.inverse_transform(ypred)
            [x] = ypred
            letters.append(x)
    return letters, image

def get_word(letters):
    word = "".join(letters)
    return word

## MODEL FOR NUMBERS

In [4]:
#CREATE TRAIN DATA
dir = "/Users/mac/Desktop/archive/Train"
train_data_num = []
img_size = 32
non_chars = ["#","$","@","&","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]

for i in os.listdir(dir):
    if i in non_chars:
        continue
    file_path = os.path.join(dir, i)
    if not os.path.isdir(file_path):
        continue
    count = 0
    for j in os.listdir(file_path):
        count += 1
        if count > 4000:
            break
        img_path = os.path.join(file_path, j)
        if not os.path.isfile(img_path):
            continue
        _, ext = os.path.splitext(img_path)
        if ext.lower() == '.png':
            img = Image.open(img_path)
            img = img.convert('RGB') 
            img_path = img_path.replace('.png', '.jpg')
            img.save(img_path, 'JPEG')
        img = cv2.imread(img_path, 0)
        if img is None:
            continue
        img = cv2.resize(img, (img_size, img_size))
        if i == "_":
            train_data_num.append([img, ":"])
        elif i== ":":
            train_data_num.append([img,"/"])
        else:
            train_data_num.append([img, i])

#CREATE VALIDATION DATA
val_dir = "/Users/mac/Desktop/archive/Validation"
val_data_num = []
img_size = 32

for i in os.listdir(val_dir):
    if i in non_chars:
        continue
    file_path = os.path.join(val_dir, i)
    if not os.path.isdir(file_path):
        continue
    count = 0
    for j in os.listdir(file_path):
        count += 1
        if count > 1000:
            break
        img_path = os.path.join(file_path, j)
        if not os.path.isfile(img_path):
            continue
        _, ext = os.path.splitext(img_path)
        if ext.lower() == '.png':
            img = Image.open(img_path)
            img = img.convert('RGB')
            img_path = img_path.replace('.png', '.jpg')
            img.save(img_path, 'JPEG')
        try:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (img_size, img_size))
        except Exception as e:
            print(f"Error processing image: {img_path}")
            continue
        if i == "_":
            val_data_num.append([img, ":"])
        elif i== ":":
            val_data_num.append([img,"/"])
        else:
            val_data_num.append([img, i])

random.shuffle(train_data_num)
random.shuffle(val_data_num)

##### SPLIT TRAIN/VAL

In [5]:
train_X_num = []
train_Y_num = []
for features,label in train_data_num:
    train_X_num.append(features)
    train_Y_num.append(label)

val_X_num = []
val_Y_num = []
for features,label in val_data_num:
    val_X_num.append(features)
    val_Y_num.append(label)

LB_num= LabelBinarizer()
train_Y_num = LB_num.fit_transform(train_Y_num)
val_Y_num = LB_num.fit_transform(val_Y_num)

train_X_num = np.array(train_X_num)/255.0
train_X_num = train_X_num.reshape(-1,32,32,1)
train_Y_num = np.array(train_Y_num)

val_X_num = np.array(val_X_num)/255.0
val_X_num = val_X_num.reshape(-1,32,32,1)
val_Y_num = np.array(val_Y_num)

## MODEL FOR CHARACTERS

In [6]:
#CREATE TRAIN DATA
dir = "/Users/mac/Desktop/archive/Train"
train_data_char = []
img_size = 32
non_chars = ["#","$","@","&","1","2","3","4","5","6","7","8","9","_",":"]

for i in os.listdir(dir):
    if i in non_chars:
        continue
    file_path = os.path.join(dir, i)
    if not os.path.isdir(file_path):
        continue
    count = 0
    for j in os.listdir(file_path):
        count += 1
        if count > 4000:
            break
        img_path = os.path.join(file_path, j)
        if not os.path.isfile(img_path):
            continue
        _, ext = os.path.splitext(img_path)
        if ext.lower() == '.png':
            img = Image.open(img_path)
            img = img.convert('RGB') 
            img_path = img_path.replace('.png', '.jpg')
            img.save(img_path, 'JPEG')
        img = cv2.imread(img_path, 0)
        if img is None:
            continue
        img = cv2.resize(img, (img_size, img_size))
        if i == "0":
            train_data_char.append([img, "O"])
        elif i== ":":
            train_data_char.append([img,"/"])
        else:
            train_data_char.append([img, i])

#CREATE VALIDATION DATA
val_dir = "/Users/mac/Desktop/archive/Validation"
val_data_char = []
img_size = 32

for i in os.listdir(val_dir):
    if i in non_chars:
        continue
    file_path = os.path.join(val_dir, i)
    if not os.path.isdir(file_path):
        continue
    count = 0
    for j in os.listdir(file_path):
        count += 1
        if count > 1000:
            break
        img_path = os.path.join(file_path, j)
        if not os.path.isfile(img_path):
            continue
        _, ext = os.path.splitext(img_path)
        if ext.lower() == '.png':
            img = Image.open(img_path)
            img = img.convert('RGB')
            img_path = img_path.replace('.png', '.jpg')
            img.save(img_path, 'JPEG')
        try:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (img_size, img_size))
        except Exception as e:
            print(f"Error processing image: {img_path}")
            continue
        if i == "0":
            val_data_char.append([img, "O"])
        elif i== ":":
            val_data_char.append([img,"/"])
        else:
            val_data_char.append([img, i])

random.shuffle(train_data_char)
random.shuffle(val_data_char)

Error processing image: /Users/mac/Desktop/archive/Validation/U/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/I/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/G/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/Z/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/S/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/F/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/O/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/M/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/J/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/C/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/V/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/Q/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/X/.DS_Store
Error processing image: /Users/mac/Des

##### SPLIT TRAIN/VAL

In [7]:
train_X_char = []
train_Y_char = []
for features,label in train_data_char:
    train_X_char.append(features)
    train_Y_char.append(label)

val_X_char = []
val_Y_char = []
for features,label in val_data_char:
    val_X_char.append(features)
    val_Y_char.append(label)

LB_char = LabelBinarizer()
train_Y_char = LB_char.fit_transform(train_Y_char)
val_Y_char = LB_char.fit_transform(val_Y_char)

train_X_char = np.array(train_X_char)/255.0
train_X_char = train_X_char.reshape(-1,32,32,1)
train_Y_char = np.array(train_Y_char)

val_X_char = np.array(val_X_char)/255.0
val_X_char = val_X_char.reshape(-1,32,32,1)
val_Y_char = np.array(val_Y_char)

## COMBINE MODEL

In [8]:
#CREATE TRAIN DATA
dir = "/Users/mac/Desktop/archive/Train"
train_data_combined = []
img_size = 32
non_chars = ["#","$","@"]

for i in os.listdir(dir):
    if i in non_chars:
        continue
    file_path = os.path.join(dir, i)
    if not os.path.isdir(file_path):
        continue
    count = 0
    for j in os.listdir(file_path):
        count += 1
        if count > 4000:
            break
        img_path = os.path.join(file_path, j)
        if not os.path.isfile(img_path):
            continue
        _, ext = os.path.splitext(img_path)
        if ext.lower() == '.png':
            img = Image.open(img_path)
            img = img.convert('RGB') 
            img_path = img_path.replace('.png', '.jpg')
            img.save(img_path, 'JPEG')
        img = cv2.imread(img_path, 0)
        if img is None:
            continue
        img = cv2.resize(img, (img_size, img_size))
        if i == "_":
            train_data_combined.append([img, ":"])
        elif i== ":":
            train_data_combined.append([img,"/"])
        else:
            train_data_combined.append([img, i])

#CREATE VALIDATION DATA
val_dir = "/Users/mac/Desktop/archive/Validation"
val_data_combined = []
img_size = 32

for i in os.listdir(val_dir):
    if i in non_chars:
        continue
    file_path = os.path.join(val_dir, i)
    if not os.path.isdir(file_path):
        continue
    count = 0
    for j in os.listdir(file_path):
        count += 1
        if count > 1000:
            break
        img_path = os.path.join(file_path, j)
        if not os.path.isfile(img_path):
            continue
        _, ext = os.path.splitext(img_path)
        if ext.lower() == '.png':
            img = Image.open(img_path)
            img = img.convert('RGB')
            img_path = img_path.replace('.png', '.jpg')
            img.save(img_path, 'JPEG')
        try:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (img_size, img_size))
        except Exception as e:
            print(f"Error processing image: {img_path}")
            continue
        if i == "_":
            val_data_combined.append([img, ":"])
        elif i== ":":
            val_data_combined.append([img,"/"])
        else:
            val_data_combined.append([img, i])

random.shuffle(train_data_combined)
random.shuffle(val_data_combined)

Error processing image: /Users/mac/Desktop/archive/Validation/U/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/I/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/G/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/Z/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/S/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/F/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/O/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/M/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/J/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/C/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/V/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/Q/.DS_Store
Error processing image: /Users/mac/Desktop/archive/Validation/X/.DS_Store
Error processing image: /Users/mac/Des

##### SPLIT TRAIN/VAL

In [9]:
train_X_combined = []
train_Y_combined = []
for features,label in train_data_combined:
    train_X_combined.append(features)
    train_Y_combined.append(label)

val_X_combined = []
val_Y_combined = []
for features,label in val_data_combined:
    val_X_combined.append(features)
    val_Y_combined.append(label)

LB_combined = LabelBinarizer()
train_Y_combined = LB_combined.fit_transform(train_Y_combined)
val_Y_combined = LB_combined.fit_transform(val_Y_combined)

train_X_combined = np.array(train_X_combined)/255.0
train_X_combined = train_X_combined.reshape(-1,32,32,1)
train_Y_combined = np.array(train_Y_combined)

val_X_combined = np.array(val_X_combined)/255.0
val_X_combined = val_X_combined.reshape(-1,32,32,1)
val_Y_combined = np.array(val_Y_combined)

#### MODELS

In [10]:
# Cargar el modelo guardado
model_char = load_model('model_char.h5')
model_num = load_model('model_numbers.h5')
model_gen=load_model('combine_model.h5')

## FINAL OUTPUT

In [11]:
imagen = cv2.imread('ParteA.jpg')
imagen = cv2.resize(imagen, (2550, 3509))
#ZONAS VEHICULO
hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
rango_amarillo_bajo = np.array([20, 100, 100])
rango_amarillo_alto = np.array([40, 255, 255])
rango_azul_bajo = np.array([90, 100, 100])
rango_azul_alto = np.array([120, 255, 255])
mascara_amarilla = cv2.inRange(hsv, rango_amarillo_bajo, rango_amarillo_alto)
mascara_azul = cv2.inRange(hsv, rango_azul_bajo, rango_azul_alto)
kernel = np.ones((5, 5), np.uint8)
mascara_amarilla = cv2.morphologyEx(mascara_amarilla, cv2.MORPH_OPEN, kernel)
mascara_azul = cv2.morphologyEx(mascara_azul, cv2.MORPH_OPEN, kernel)
contornos_amarillos, _ = cv2.findContours(mascara_amarilla, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contornos_azules, _ = cv2.findContours(mascara_azul, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

coordenadas_rectangulos = []

for contorno in contornos_amarillos:
    area = cv2.contourArea(contorno)
    if area > 4000:  
        x, y, w, h = cv2.boundingRect(contorno)
        coordenadas_rectangulos.append(('B', x, y, w, h))

for contorno in contornos_azules:
    area = cv2.contourArea(contorno)
    if area > 4000:  
        x, y, w, h = cv2.boundingRect(contorno)
        coordenadas_rectangulos.append(('A', x, y, w, h))

df_coordenadas = pd.DataFrame(coordenadas_rectangulos, columns=['Zona', 'X', 'Y', 'W', 'H'])

#ZONA DE ARRIBA
gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
suavizado = cv2.GaussianBlur(gris, (5, 5), 0)
binaria = cv2.adaptiveThreshold(suavizado, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 1)
contornos, _ = cv2.findContours(binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
coordenadas_rectangulos = []

for i, contorno in enumerate(contornos):
    area = cv2.contourArea(contorno)
    if area > 2000:  
        x, y, w, h = cv2.boundingRect(contorno)
        if y< 400:
            coordenadas_rectangulos.append((x, y, w, h))

df_coordenadas_2 = pd.DataFrame(coordenadas_rectangulos, columns=['X', 'Y', 'W', 'H'])
df_coordenadas_2['Suma_XY'] = df_coordenadas_2['X'] + df_coordenadas_2['Y']
df_coordenadas_2 = df_coordenadas_2.sort_values('Suma_XY')
df_coordenadas_2 = df_coordenadas_2.drop('Suma_XY', axis=1)
df_coordenadas_2 = df_coordenadas_2.reset_index().sort_index()
df_coordenadas_2 = df_coordenadas_2.drop('index', axis=1)
len(df_coordenadas_2)

print("Cantidad de rectángulos EN LA ZONA DE ARRIBA:", len(df_coordenadas_2))
print("Cantidad de rectángulos amarillos:", len(df_coordenadas[df_coordenadas['Zona'] == 'B']))
print("Cantidad de rectángulos azules:", len(df_coordenadas[df_coordenadas['Zona'] == 'A']))

Cantidad de rectángulos EN LA ZONA DE ARRIBA: 5
Cantidad de rectángulos amarillos: 1
Cantidad de rectángulos azules: 1


In [12]:
def preprocess_image(image):
    if image is None or image.size == 0:
        return None
    # Redimensionar l imagen para mejorar la resolución
    resized = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Aplicar un enfoque para mejorar la nitidez
    sharpened = cv2.detailEnhance(resized, sigma_s=20, sigma_r=2)
    
    # Retornar la imagen preprocesada
    return sharpened

In [13]:
results = pd.read_excel("template.xlsx")
image = cv2.imread('ParteA.jpg')
image = cv2.resize(image, (2550, 3509))
#VEHICULOS
for index, row in df_coordenadas.iterrows():
    if row["Zona"]=="A":
        x1 = row["X"] 
        y1 = row["Y"] 
        w1 = row["W"]
        h1 = row["H"]
        cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)
        roi = image[y1:y1+h1, x1:x1+w1]
        roi = cv2.resize(roi, (1185, 3070))
        for i, r in results.iterrows():
            if i in [11, 12, 13, 14, 15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94]:
                x = int(results.at[i, 'x'])
                y = int(results.at[i, 'y'])
                w = int(results.at[i, 'w'])
                h = int(results.at[i, 'h'])
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                if i in [14,16,23,24,25,26,31,36,40,42,94]:
                    letters, image_with_boxes = get_numbers(preprocessed_image)
                elif i not in[14,16,23,24,25,26,31,36,40,42,94]:
                    if i in [77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93]:
                        letters, image_with_boxes = get_letters(preprocessed_image)
                        if len(letters)>0:
                            letters="X"
                    else:
                        letters, image_with_boxes = get_combined(preprocessed_image)
                word = get_word(letters)
                results.at[i, 'text'] = word
    else:
        x1 = row["X"] 
        y1 = row["Y"] 
        w1 = row["W"]
        h1 = row["H"]
        cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)
        roi = image[y1:y1+h1, x1:x1+w1]
        roi = cv2.resize(roi, (1185, 3070))
        for i, r in results.iterrows():
            if i in [44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112]:
                x = int(results.at[i, 'x'])
                y = int(results.at[i, 'y'])
                w = int(results.at[i, 'w'])
                h = int(results.at[i, 'h'])
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                if i in [47,49,56,57,58,59,64,69,73,75,95]:
                    letters, image_with_boxes = get_numbers(preprocessed_image)
                elif i not in[47,49,56,57,58,59,64,69,73,75,95]:
                    if i in [96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112]:
                        letters, image_with_boxes = get_letters(preprocessed_image)
                        if len(letters)>0:
                            letters="X"
                    else:
                        letters, image_with_boxes = get_combined(preprocessed_image)
                word = get_word(letters)
                results.at[i, 'text'] = word


#PARTE DE ARRIBA
for index, row in df_coordenadas_2.iterrows():
    x1 = row["X"] 
    y1 = row["Y"] 
    w1 = row["W"]
    h1 = row["H"]
    cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)
    roi = image[y1:y1+h1, x1:x1+w1]
    for i, r in results.iterrows():
        x = int(results.at[i, 'x'])
        y = int(results.at[i, 'y'])
        w = int(results.at[i, 'w'])
        h = int(results.at[i, 'h'])
        if index==0:
            if i in [0,1]:
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                letters, image_with_boxes = get_numbers(preprocessed_image)
                word = get_word(letters)
                results.at[i, 'text'] = word
        elif index==1:
            if i in [3,4,5,6]:
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                letters, image_with_boxes = get_letters(preprocessed_image)
                if len(letters)>0:
                    letters="X"
                word = get_word(letters)
                results.at[i, 'text'] = word
        elif index==2:
            if i in [2,7]:
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                letters, image_with_boxes = get_letters(preprocessed_image)
                word = get_word(letters)
                results.at[i, 'text'] = word
        elif index==3:
            if i in [10]:
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                letters, image_with_boxes = get_combined(preprocessed_image)
                word = get_word(letters)
                results.at[i, 'text'] = word
        elif index==4:
            if i in [8,9]:
                roi2=roi[y:y+h, x:x+w]
                preprocessed_image = preprocess_image(roi2)
                letters, image_with_boxes = get_letters(preprocessed_image)
                if len(letters)>0:
                    letters="X"
                word = get_word(letters)
                results.at[i, 'text'] = word

1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [==============================] - 0s 11ms/step
Error: No contours found
Error: No contours found
Error: No contours found
Error: No contours found
Error: No contours found
1/1 [==============================] - 0s 11ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
Error: No contours found
1/1 [==============================] - 0s 11ms/step
Error: No contours found
1/1 [==============================] - 0s 12ms/step
Error: No contours found
1/1 [=======================